## DS 7337 - Natural Language Processing

### Author: Brandon Croom

### Homework: 2

In [18]:
# import nltk and other items
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.book import *
from nltk.corpus import words

# define the text word count
def text_word_count(text_data):
    return len(text_data)

# define the text vocabulary size
def text_vocab_size(text_data):
    return len(set(text_data))

# define a method that will get us all words in the english language
def english_lang_words():
    word_list = words.words()
    return len(word_list)

#### 1.	In Python, create a method for scoring the vocabulary size of a text, and normalize the score from 0 to 1. It does not matter what method you use for normalization as long as you explain it in a short paragraph. (Various methods will be discussed in the live session.)

In [20]:
# define the vocabulary size method. Filter out everything that is not alpha numeric and make it all lower case.
# We'll normalize using the size calculation of the vocabulary and divide by the total number of the vocabulary size. Using the standard normalization formula (x - min(x) / (max(x) - min(x))). Assuming min(x) = 0 (this is the fewest number of words possible) this minimizes to x/max(x) where x=size of the text after cleansing and max(x) = the full size of the english language 
def vocab_size(text):
    size = len(set(word.lower() for word in text if word.isalpha()))
    return size/english_lang_words()
    #text_vocab_size(text)

vocab_size(text1)

0.07159029467423628

### 2.	After consulting section 3.2 in chapter 1 of Bird-Klein, create a method for scoring the long-word vocabulary size of a text, and likewise normalize (and explain) the scoring as in step 1 above.

In [22]:
# define the long word score function. We'll allow the user to input the word length and the number of times the word should occur in the text before it's counted. We'll normalize by 
# dividing the length of the long words by the total number of words in the text provided. Using the standard normalization formula (x - min(x) / (max(x) - min(x))). Assuming min(x) = 0 (this is the fewest number of words possible) this minimizes to x/max(x) where x=size of the text after cleansing and max(x) = the full size of the english language 
def long_word_score(text, word_len=7, word_freq=7):
    fdist = FreqDist(text)
    size = sorted(word for word in set(text) if len(word) > word_len and fdist[word] > word_freq)
    return len(size)/english_lang_words()

long_word_score(text5)

8.025817788591511e-05

### 3.	Now create a “text difficulty score” by combining the lexical diversity score from homework 1, and your normalized score of vocabulary size and long-word vocabulary size, in equal weighting. Explain what you see when this score is applied to same graded texts you used in homework 1.

In [23]:
#HW 1 lexical diversity function
def lexical_diversity(text_data):
    word_count = text_word_count(text_data)
    vocab_size = text_vocab_size(text_data)
    diversity_score = vocab_size / word_count
    return diversity_score

# text difficulty score function. Create a weighted average function. Allow the user to input specific weights for
# lexical diversity, word score, and vocabulary size. Defaults for all are 1 to keep the weights equal
def text_diff_score(text, lex_div_weight=1,word_score_weight=1,vocab_size_weight=1):
    lex_div = lexical_diversity(text)*lex_div_weight
    word_score = long_word_score(text) * word_score_weight
    vsize = vocab_size(text) * vocab_size_weight

    return (lex_div + word_score + vsize)/3
    

# load the same corpus used in HW1
corpus_root = "C://RAI//DS7337-NLP//HW1//corpus"
file_pattern = r".*/*.*\.txt"
ptb = PlaintextCorpusReader(corpus_root,file_pattern)
ptb.fileids()

# print out the new text difficulty scores
print("Text Difficulty Score The Ontario Readers: Third Book: ",text_diff_score(ptb.words(ptb.fileids()[0])))
print("Text Difficulty Score The Ontario Readers: Fourth Book: ",text_diff_score(ptb.words(ptb.fileids()[1])))
print("Text Difficulty Score The Ontario Readers: The High School Book: ",text_diff_score(ptb.words(ptb.fileids()[2])))

Text Difficulty Score The Ontario Readers: Third Book:  0.04969884364250884
Text Difficulty Score The Ontario Readers: Fourth Book:  0.05454543644267229
Text Difficulty Score The Ontario Readers: The High School Book:  0.05382365014314133



#### Using the text difficult score above, we see that this text difficulty score follows a similar pattern to just looking at lexical diversity for the texts, like in HW1. It's a bit counter intuitive. The text designed for high school has a score of 0.053, which is lower than the book for fourth grade. We would expect that the high school book would have the higher of the scores when compared to books for third and fourth graders. The text difficulty scores for the third and fourth grade books show, they are close in difficulty, but do have a separation which would be indicative of the grade change.
